In [ ]:
'''
K-Means Clustering
'''

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('Live.csv')
print(df.shape)
df.head()

In [ ]:
'''
Мы видим, что в наборе данных 7050 экземпляров и 16 атрибутов. 
В описании набора данных указано, что в наборе данных 7051 экземпляр и 12 атрибутов.

Итак, мы можем сделать вывод, что первый экземпляр — это заголовок строки, и в наборе данных есть 4 дополнительных атрибута.
'''

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
'''
Мы видим, что в наборе данных есть 4 избыточных столбца. Нам следует отбросить их, прежде чем двигаться дальше.
'''

In [ ]:
df.drop(['Column1', 'Column2', 'Column3', 'Column4'], axis=1, inplace=True)
df.info()

In [ ]:
'''
Мы видим, что есть 3 символьные переменные (тип данных = объект) и оставшиеся 9 числовых переменных (тип данных = int64).
'''

In [ ]:
# Просмотр статистической сводки числовых переменных

df.describe()

In [ ]:
'''
В наборе данных есть 3 категориальные переменные. Будем исследовать их одину за другой.
'''

In [ ]:
len(df['status_id'].unique())

In [ ]:
'''
Мы видим, что в переменной status_id 6997 уникальных меток. 
Общее количество экземпляров в наборе данных — 7050. 
То есть это уникальный идентификатор для каждого из экземпляров. 
Таким образом, это не та переменная, которую мы можем использовать.
'''

In [ ]:
df['status_published'].unique()

In [ ]:
len(df['status_published'].unique())

In [ ]:
'''
Опять же, мы видим, что в переменной status_published 6913 уникальных меток. 
Общее количество экземпляров в наборе данных — 7050. 
То есть это еще и примерно уникальный идентификатор для каждого из экземпляров. 
Таким образом, это не та переменная, которую мы можем использовать.
'''

In [ ]:
df['status_type'].unique()

In [ ]:
len(df['status_type'].unique())

In [ ]:
'''
Мы видим, что в переменной status_type есть 4 категории меток.
Удалим переменные status_id и status_published из набора данных.
'''

In [ ]:
df.drop(['status_id', 'status_published'], axis=1, inplace=True)

df.info()

In [ ]:
df.head()

In [ ]:
'''
Мы видим, что в наборе данных есть 1 нечисловой столбец status_type. Преобразуем его в целочисленные эквиваленты.
'''

In [ ]:
X = df
y = df['status_type']

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

X['status_type'] = le.fit_transform(X['status_type'])
y = le.transform(y)

X.info()

In [ ]:
X.head()

In [ ]:
# Масштабирование признаков

cols = X.columns

In [ ]:
from sklearn.preprocessing import MinMaxScaler

ms = MinMaxScaler()
X = ms.fit_transform(X)

X = pd.DataFrame(X, columns=[cols])

X.head()

In [ ]:
'''
Модель K-Means с двумя кластерами
'''

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=0) 

kmeans.fit(X)

In [ ]:
'''
Исследование параметров модели K-Means
'''

In [ ]:
kmeans.cluster_centers_

In [ ]:
'''
Алгоритм KMeans группирует данные, пытаясь разделить выборки на n групп с равными дисперсиями, минимизируя критерий, 
известный как инерция, или критерий суммы квадратов внутри кластера. Инерция, или критерий суммы квадратов внутри кластера, 
может быть признана мера того, насколько внутренне согласованы кластеры.
Алгоритм k-means делит набор из N выборок X на K непересекающихся кластеров C, 
каждый из которых описывается средним значением j выборок в кластере. Эти средства обычно называют центроидами кластера.
Алгоритм K-Means направлен на выбор центроидов, которые минимизируют инерцию, или сумму квадратов критерия внутри кластера.

Инерция
Инерция не является нормализованным показателем.

Более низкие значения инерции лучше, а ноль является оптимальным.

Но в очень многомерных пространствах евклидовы расстояния имеют тенденцию раздуваться (это пример проклятия размерности).

Запуск алгоритма уменьшения размерности, такого как PCA, перед кластеризацией k-means может облегчить эту проблему 
и ускорить вычисления.
'''

In [ ]:
kmeans.inertia_

In [ ]:
'''
Чем меньше инерция модели, тем лучше она подходит.
Мы видим, что модель имеет очень высокую инерционность. 
Таким образом, это не очень хорошая модель, соответствующая данным.
'''

In [ ]:
# Проверка качества слабой классификации по модели

labels = kmeans.labels_

## проверьте, сколько образцов было правильно маркированоeled
correct_labels = sum(y == labels)

print("Результат: %d из %d образцов были правильно маркированы." % (correct_labels, y.size))

In [ ]:
'''
С помощью нашей неконтролируемой модели мы достигли слабой точности классификации в 1%.
'''

In [ ]:
'''
Используем elbow method, чтобы найти оптимальное количество кластеров.
'''

In [ ]:
from sklearn.cluster import KMeans
cs = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
    kmeans.fit(X)
    cs.append(kmeans.inertia_)
plt.plot(range(1, 11), cs)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('CS')
plt.show()

In [ ]:
'''
На приведенном выше графике мы видим, что при k = 2 имеется излом.
Следовательно, k=2 можно считать подходящим номером кластера для кластеризации этих данных.
Но мы видели, что я достиг слабой точности классификации в 1% при k = 2.
Для удобства я снова напишу требуемый код с k=2.
'''

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2,random_state=0)

kmeans.fit(X)

labels = kmeans.labels_

correct_labels = sum(y == labels)

print("Результат: %d из %d образцов были правильно маркированы." % (correct_labels, y.size))

print('Оценка точности: {0:0.2f}'. format(correct_labels/float(y.size)))

In [ ]:
'''
Таким образом, наша слабая модель неконтролируемой классификации достигла очень низкой точности классификации — 1%.
Проверим точность модели на разном количестве кластеров.
'''

In [ ]:
# Модель KMeans с 3 кластерами

kmeans = KMeans(n_clusters=3, random_state=0)

kmeans.fit(X)

labels = kmeans.labels_

correct_labels = sum(y == labels)
print("Результат: %d из %d образцов были правильно маркированы." % (correct_labels, y.size))
print('Оценка точности: {0:0.2f}'. format(correct_labels/float(y.size)))

In [ ]:
# Модель KMeans с 4 кластерами

kmeans = KMeans(n_clusters=4, random_state=0)

kmeans.fit(X)

labels = kmeans.labels_

correct_labels = sum(y == labels)
print("Результат: %d из %d образцов были правильно маркированы." % (correct_labels, y.size))
print('Оценка точности: {0:0.2f}'. format(correct_labels/float(y.size)))

In [ ]:
# Модель KMeans с 5 кластерами

kmeans = KMeans(n_clusters=5, random_state=0)

kmeans.fit(X)

labels = kmeans.labels_

correct_labels = sum(y == labels)
print("Результат: %d из %d образцов были правильно маркированы." % (correct_labels, y.size))
print('Оценка точности: {0:0.2f}'. format(correct_labels/float(y.size)))

In [ ]:
'''
Мы достигли относительно высокой точности 62% при k=4.
При k=5 точность снизилась.
'''

In [ ]:
'''
В этом проекте я реализовал самый популярный метод неконтролируемой кластеризации, называемый кластеризацией K-Means.

Я применил elbow method и обнаружил, что k = 2 (k — количество кластеров) можно считать подходящим количеством 
кластеров для кластеризации этих данных.

Я обнаружил, что модель имеет очень высокую инерцию — 237,7572. 
Таким образом, это не очень хорошая модель, соответствующая данным.

С помощью нашей неконтролируемой модели я достиг слабой точности классификации в 1% при k=2.

Итак, я изменил значение k и обнаружил относительно более высокую точность классификации - 62% при k = 4.

При этом при k = 5, точность кпала до 1%.

Следовательно, мы можем заключить, что k = 4 — оптимальное количество кластеров
'''